In [ ]:
!pip install folium
!pip install geopandas

In [24]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import json
import requests

### 530 bus dataset 


In [14]:
app_id = "110703014-6dcbb76e-11e4-46d4"
app_key = "44ecf82b-641e-4a9d-859b-07f708481690"

In [30]:
auth_url="https://tdx.transportdata.tw/auth/realms/TDXConnect/protocol/openid-connect/token"
shape_url = "https://tdx.transportdata.tw/api/basic/v2/Bus/Shape/City/Taipei/530?%24top=30&%24format=JSON"
stop_url = "https://tdx.transportdata.tw/api/basic/v2/Bus/RealTimeByFrequency/City/Taipei/530?%24top=30&%24format=JSON"

In [29]:
class Auth():

    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key

    def get_auth_header(self):
        content_type = 'application/x-www-form-urlencoded'
        grant_type = 'client_credentials'

        return{
            'content-type' : content_type,
            'grant_type' : grant_type,
            'client_id' : self.app_id,
            'client_secret' : self.app_key
        }

In [31]:
class data():

    def __init__(self, app_id, app_key, auth_response):
        self.app_id = app_id
        self.app_key = app_key
        self.auth_response = auth_response

    def get_data_header(self):
        auth_JSON = json.loads(self.auth_response.text)
        access_token = auth_JSON.get('access_token')

        return{
            'authorization': 'Bearer ' + access_token,
            'Accept-Encoding': 'gzip'
        }

In [32]:
a = Auth(app_id, app_key)
auth_response = requests.post(auth_url, a.get_auth_header())
d = data(app_id, app_key, auth_response)
data_response = requests.get(shape_url, headers=d.get_data_header())

In [33]:
with open("shape.json", "w", encoding="utf-8") as f:
        json.dump(data_response.text, f, ensure_ascii=False)

In [34]:
a = Auth(app_id, app_key)
auth_response = requests.post(auth_url, a.get_auth_header())
d = data(app_id, app_key, auth_response)
data_response = requests.get(stop_url, headers=d.get_data_header())

In [35]:
with open("stop.json", "w", encoding="utf-8") as f:
        json.dump(data_response.text, f, ensure_ascii=False)

### draw map

In [13]:
Taipei_map = folium.Map(location=[25.03949721583935, 121.55957402734612], tiles='openstreetmap', zoom_start=15)
Taipei_map